# 🔍 Fun with Keyword Search using VideoDB

<a href="https://colab.research.google.com/github/video-db/videodb-cookbook/blob/main/examples/Keyword_Search_1`.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 💬 Overview
--- 


In this tutorial, let’s explore the powerful functionality of Keyword Search in VideoDB. This feature enables users to efficiently locate any keyword or phrase within their video assets, streamlining the process of content discovery.

![](https://raw.githubusercontent.com/video-db/videodb-cookbook-assets/main/images/examples/Keyword_Search_1.png)

![](https://raw.githubusercontent.com/video-db/videodb-cookbook/main/images/Intro_Outro_Inline/image.png)

## Setup
---

### 📦  Installing packages 

In [ ]:
%pip install videodb

### 🔑 API Keys
Before proceeding, ensure access to [VideoDB](https://videodb.io) API key. 

> Get your API key from [VideoDB Console](https://console.videodb.io). ( Free for first 50 uploads, **No credit card required** ) 🎉

In [3]:
import os

os.environ["VIDEO_DB_API_KEY"] = ""

## Steps
---

### 🌐 Step 1: Connect to VideoDB
Begin by establishing a connection to VideoDB using your API key

In [4]:
from videodb import connect

# Connect to VideoDB using your API key
conn = connect()
coll = conn.get_collection()

### 🎥 Step 2: Upload Videos

Upload the video to VideoDB collection. You can upload the video asset from your local device or from a YouTube URL to upload the video from its source.

In [5]:
video = coll.upload(url="https://www.youtube.com/watch?v=Uvufun6xer8")
video.play()

'https://console.videodb.io/player?url=https://stream.videodb.io/v3/published/manifests/7d4fe7ce-32f9-4869-8c0a-f8961aa3252f.m3u8'

### 🔊  Step 3: Index Spoken Words

Index the spoken words in your video to enable  keyword search.


In [6]:
video.index_spoken_words()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:10<00:00,  1.42it/s]


### 🔎  Step 4: Search for any keyword

Utilize the keyword search function by specifying your desired keyword as input.

In [7]:
from videodb import SearchType

result = video.search(query='metaverse', search_type=SearchType.keyword)

### 👀 Step 5: Preview and Share

Preview your video with a compilation of all the clips matching your search query.  You can access the stream link alongside the preview to share the Keyword Search result with others.

In [8]:
result.play()

'https://console.videodb.io/player?url=https://stream.videodb.io/v3/published/manifests/8ebd41a8-12d3-4422-9e49-628f5db41133.m3u8'

## 🎉 Conclusion
---
With VideoDB, you can harness the power of Keyword Search and refine your results with padding control. Simply by connecting to VideoDB, and utilizing the search function, you can easily discover relevant content within your video collection. With the added flexibility of setting padding duration, you ensure thorough coverage around keyword occurrences. Keep exploring and innovating with VideoDB to unlock even more possibilities for your projects!


For more such explorations, refer to the [VideoDB Docs](https://docs.videodb.io/) and join the VideoDB community on [GitHub](https://github.com/video-db) or [Discord](https://discord.com/invite/py9P639jGz) for support and collaboration.